In [1]:
import pandas as pd

# Function to downcast numeric columns to save memory
def downcast(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

# Load the CSV file using pandas
columns_to_use = [
    '70000', 'D', 'N', 'F', 'MK15 9HP', 'WILLEN', 'MILTON KEYNES', 
    'MILTON KEYNES.1', 'MILTON KEYNES.2', '1995-07-07 00:00', 'A'
]

df = pd.read_csv('C:/Users/Guest01/Documents/Manpreet_thesis/Datasets/UK_property_price/UKarchive/202304.csv', usecols=columns_to_use)

# Rename the columns for ease of use
df = df.rename(columns={
    '70000': 'price', 
    'D': 'Property_Type', 
    'N': 'Old/New', 
    'F': 'Duration', 
    'MK15 9HP': 'Postcode', 
    'WILLEN': 'Locality', 
    'MILTON KEYNES': 'Town/City', 
    'MILTON KEYNES.1': 'District', 
    'MILTON KEYNES.2': 'County', 
    '1995-07-07 00:00': 'Date_of_Transfer', 
    'A': 'PPDCategory_Type'
})

# Apply downcasting to reduce memory usage immediately
df = downcast(df)

# Stratified sampling to reduce dataset size (75% sampling by 'Property_Type' and 'Town/City')
df_sampled = df.groupby(['Property_Type', 'Town/City'], group_keys=False).apply(lambda x: x.sample(frac=0.75, random_state=42)).reset_index(drop=True)

# Filter the dataset to include only standard residential sales (PPDCategory_Type 'A')
df_sampled = df_sampled[df_sampled['PPDCategory_Type'] == 'A']


C:\Users\Guest01\AppData\Local\Temp\ipykernel_16864\2973047249.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df.groupby(['Property_Type', 'Town/City'], group_keys=False).apply(lambda x: x.sample(frac=0.75, random_state=42)).reset_index(drop=True)


In [2]:
# Fill missing values for categorical columns
categorical_columns = ['Property_Type', 'Old/New', 'Duration', 'Postcode', 'Locality', 'Town/City', 'District', 'County', 'PPDCategory_Type']
mode_values = {col: df_sampled[col].mode()[0] for col in categorical_columns}
df_sampled = df_sampled.fillna(mode_values)

# Fill missing 'price' values (numeric) with the median
df_sampled['price'] = df_sampled['price'].fillna(df_sampled['price'].median())

# Extract date features
df_sampled['Date_of_Transfer'] = pd.to_datetime(df_sampled['Date_of_Transfer'])
df_sampled['Year'] = df_sampled['Date_of_Transfer'].dt.year
df_sampled['Month'] = df_sampled['Date_of_Transfer'].dt.month
df_sampled['Day'] = df_sampled['Date_of_Transfer'].dt.day
df_sampled['DayOfWeek'] = df_sampled['Date_of_Transfer'].dt.dayofweek

# Drop the raw date column
df_sampled = df_sampled.drop(columns=['Date_of_Transfer'])


In [ ]:
# Function to apply frequency encoding for categorical variables
def optimized_frequency_encoding(df, categorical_columns):
    for col in categorical_columns:
        if df[col].dtype == 'object':
            freq_encoding = df[col].value_counts().to_dict()
            df[col + '_freq'] = df[col].map(freq_encoding)
            df = df.drop(columns=[col])  # Drop the original object column
    return df

# List of categorical columns
categorical_columns = ['Postcode', 'Property_Type', 'Old/New', 'Duration', 'Locality', 'Town/City', 'District', 'County', 'PPDCategory_Type']

# Apply frequency encoding to all categorical columns
df_sampled = optimized_frequency_encoding(df_sampled, categorical_columns)

# Unified downcast function for all numeric columns
def downcast_all_numeric(df):
    df[df.select_dtypes(include=['float64']).columns] = df.select_dtypes(include=['float64']).apply(pd.to_numeric, downcast='float')
    df[df.select_dtypes(include=['int64']).columns] = df.select_dtypes(include=['int64']).apply(pd.to_numeric, downcast='integer')
    return df

# Downcast all numeric columns (including frequency-encoded columns)
df_sampled = downcast_all_numeric(df_sampled)

In [ ]:
# from category_encoders import TargetEncoder

# # Apply encoding to the categorical columns
# categorical_columns = ['Property_Type', 'Old/New', 'Duration', 'Postcode', 'Locality', 'Town/City', 'District', 'County']
# encoder = TargetEncoder(cols=categorical_columns)

# # Apply the target encoder to the categorical columns
# df_sampled[categorical_columns] = encoder.fit_transform(df_sampled[categorical_columns], df_sampled['price'])


In [4]:
# Define X and y
X = df_sampled.drop(columns=['price', 'PPDCategory_Type'])  # PPDCategory_Type is not needed for prediction
y = df_sampled['price']

# Check if any non-numeric columns are still present
non_numeric_columns_in_X = X.select_dtypes(include=['object']).columns
print("Non-numeric columns in X:", non_numeric_columns_in_X)  # Should print an empty index

# Print the shape of X and y
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")


Non-numeric columns in X: Index([], dtype='object')
Shape of X: (20274118, 12)
Shape of y: (20274118,)


In [5]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil

# Function to calculate and return metrics for Linear Regression
def calculate_metrics(X_train, X_test, y_train, y_test):
    lr = LinearRegression()

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'fit_intercept': [True, False]
    }

    random_search = RandomizedSearchCV(lr, param_distributions, n_iter=2, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        #'MSE': mse,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }

In [7]:
import gc # Garbage Collector

# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Call garbage collection after each iteration to free up memory
        gc.collect()    

        # Accumulate total       
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")

Metrics for sample size 1.0:
RMSE: 179447.2191940169
MAPE: 0.5864803383779877
R2: 0.48288769538631215
nRMSE: 0.0019938580131986104
Execution Time (Raw): 83.70704317092896
Normalized Time (s/MB): 0.04924609554344822
Average CPU Usage: 30.299999999999997
Sample Size: 1.0
--------------------------------------------------
Metrics for sample size 0.5:
RMSE: 172358.08757405242
MAPE: 0.6572156649855813
R2: 0.5135057036191536
nRMSE: 0.003077823047354919
Execution Time (Raw): 19.989565134048462
Normalized Time (s/MB): 0.023496794208709395
Average CPU Usage: 40.1
Sample Size: 0.5
--------------------------------------------------
Metrics for sample size 0.25:
RMSE: 172364.27566053183
MAPE: 0.657871076489272
R2: 0.5134707702677594
nRMSE: 0.0032302151216406848
Execution Time (Raw): 10.047922849655151
Normalized Time (s/MB): 0.02362172434770791
Average CPU Usage: 44.800000000000004
Sample Size: 0.25
--------------------------------------------------
Metrics for sample size 0.125:
RMSE: 172401.2617

In [8]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df

Total Execution Time for Entire Process (Raw): 1 minutes and 59.47 seconds
Total Normalized Execution Time for Entire Process: 0.03737970 seconds per MB
Total Average CPU Usage for Entire Process: 28.01%


,RMSE,MAPE,R2,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,179447.219194,0.586480,0.482888,0.001994,83.707043,0.049246,30.30,1.000
1,172358.087574,0.657216,0.513506,0.003078,19.989565,0.023497,40.10,0.500
2,172364.275661,0.657871,0.513471,0.003230,10.047923,0.023622,44.80,0.250
3,172401.261748,0.662213,0.513262,0.003323,5.247692,0.024674,43.25,0.125
4,70721.691757,0.367152,0.698931,0.073554,0.050152,7.469886,16.45,100.000
5,100495.624138,0.501598,0.555612,0.047471,0.035365,0.526747,15.75,1000.000
6,131533.703748,0.631848,0.593057,0.009432,0.118859,0.177035,19.15,10000.000
7,144744.583616,0.652888,0.543348,0.006579,0.269567,0.040151,14.25,100000.000
